# Replication of the Experiment on Comparing Static and Dynamic Weighted Software Coupling Metrics

This notebook contains all scripts for the replication of the experiment conducted by Schnoor and Hasselbring: Schnoor, H. & Hasselbring, W., 2020. Comparing Static and Dynamic Weighted Software Coupling Metrics. Computers, 9(2), P. 24. Available at: https://www.mdpi.com/2073-431X/9/2/24.

The differences of the coupling metrics are studied by comparing the ranking obtained by ordering the program modules by their coupling degree using the Kendall-Tau distance. The coupling degrees (import = outgoing calls, export = incoming calls, combined = import + export) are obtained from static and dynamic dependency graphs stored in a Neo4j database.

The results of the replication study are presented using the following triple $\alpha : \beta1 \leftrightarrow \beta2 $, where
* $\alpha$ is $c$ or $p$ expressing **c**lass or **p**ackage coupling,
* $\beta1$ is $s$ or $u$ expressing whether the left-hand side analysis is **s**tatic or (dynamic) **u**nweighted,
* $\beta2$ is $u$ or $w$ expressing whether the right-hand side analysis is (dynamic) **u**nweighted or (dynamic) **w**eighted.




## Database connection
Establish the connection to the Neo4j graph database containing the static and dynamic dependency graphs.

In [ ]:
import py2neo

graph = py2neo.Graph(host='localhost', user='neo4j', password='neo4j')

## Kendall-Tau distance

For a finite base set $S$ with size $n$, the metric compares two linear orders $<1$ and $<2$. The Kendall–Tau distance $\tau(<1, <2)$ is the number of swaps needed to obtain the order $<1$ from $<2$, normalized by dividing by number of possible swaps $\frac{n \cdot (n−1)}{2}$. Hence, $τ(<1, <2)$ is always between $0$ (if $<1$ and $<2$ are identical)
and $1$ (if $<1$ is the reverse of $<2$). Values smaller than $0.5$ indicate that the orders are closer together than expected from two random orders, while values larger than $0.5$ indicate the opposite. Values further away from $0.5$ imply higher correlation between two orders.

In [ ]:
import numpy as np
import pandas as pd

try:
    # Python 2
    xrange
except NameError:
    # Python 3, xrange is now named range
    xrange = range

def kendalltau_distance(df1, df2, column, level, normalized=True):
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    # create rank column for import|export|combined
    df1_copy['rank']=df1_copy[column].rank(ascending=True, method='average')
    df2_copy['rank']=df2_copy[column].rank(ascending=True, method='average')

    # sort values according to rank and class|package name
    df1_copy.sort_values(by=['rank',level], inplace=True, ascending=[False, False])
    df2_copy.sort_values(by=['rank', level], inplace=True, ascending=[False, False])

    # calculate Kendall-Tau distance
    x = np.asarray(df1_copy.index.values.tolist())
    y = np.asarray(df2_copy.index.values.tolist())
    kendalltau_distance = 0
    n = x.size - 1
    weights = np.ones(n)   
    for i in xrange(n - 1, -1, -1):
        key = x[i]
        j = i + 1
        while j <= n and np.where(y == key)[0] > np.where(y == x[j])[0]:
            x[j - 1] = x[j]
            kendalltau_distance += weights[j - 1]
            j += 1
        x[j - 1] = key 
    if normalized:
        n = len(x)
        return kendalltau_distance / ((n * (n - 1)) / 2)  
    return kendalltau_distance

## Query coupling metrics and compare coupling orders
Execute Cypher queries to calculate the dependency graph metrics, store the results in dataframes, and compare the coupling orders using the Kendall-Tau distance.



### $c : s \leftrightarrow u$

In [ ]:
# Query coupling degrees from static dependency graph on class level
query_c_s ="""
MATCH (type:Type:Java:Experiment)
WITH type.fqn AS class
OPTIONAL MATCH (:Type:Java:Experiment{fqn:class})-[out:CALLS]->(callee:Type:Experiment)
WITH class, COUNT(DISTINCT callee.fqn) AS import
OPTIONAL MATCH (caller:Type:Experiment)-[in:CALLS]->(:Type:Java:Experiment{fqn:class})
WITH class, import, COUNT(DISTINCT caller.fqn) AS export
WHERE import + export > 0
RETURN class, import, export, SUM(import + export) AS combined
ORDER BY class
"""
df_c_s = pd.DataFrame(graph.run(query_c_s).data())

# Query coupling degrees from unweighted dynamic dependency graph on class level
query_c_u ="""
MATCH (type:Type:Kieker:Experiment)
OPTIONAL MATCH (type)-[out:CALLS]->(:Type:Kieker:Experiment)
WITH type, COUNT(out) AS import
OPTIONAL MATCH (:Type:Kieker:Experiment)-[in:CALLS]->(type)
WITH type, import, COUNT(in) as export
WHERE import + export > 0
RETURN type.fqn AS class, import, export, (import+export) AS combined
ORDER BY class
"""
df_c_u = pd.DataFrame(graph.run(query_c_u).data())

# Compare coupling orders
c_s_u_i = kendalltau_distance(df_c_s, df_c_u, 'import', 'class')
c_s_u_e = kendalltau_distance(df_c_s, df_c_u, 'export', 'class')
c_s_u_c = kendalltau_distance(df_c_s, df_c_u, 'combined', 'class')

print('replication c : s <-> u: import =', format(round(c_s_u_i, 2), '.2f'), 'export =', format(round(c_s_u_e, 2), '.2f'),
      'combined =', format(round(c_s_u_c, 2), '.2f'), 'average =', format(round((c_s_u_i + c_s_u_e + c_s_u_c)/3, 2), '.2f'))

# original    (1) c : s <-> u: import = 0.31* export = 0.41 combined = 0.35
# replication (1) c : s <-> u: import = 0.32 export = 0.41 combined = 0.35
# original    (2) c : s <-> u: import = 0.30 export = 0.41 combined = 0.34 average = 0.35
# replication (2) c : s <-> u: import = 0.32 export = 0.41 combined = 0.35 average = 0.36
# original    (3) c : s <-> u: import = 0.38 export = 0.38 combined = 0.36 average = 0.37
# replication (3) c : s <-> u: import = 0.38 export = 0.38 combined = 0.36 average = 0.37
# original    (4) c : s <-> u: import = 0.37 export = 0.38 combined = 0.35 average = 0.37
# replication (4) c : s <-> u: import = 0.37 export = 0.38 combined = 0.35 average = 0.37

### $c : s \leftrightarrow w$

In [ ]:
# Query coupling degrees from static dependency graph on class level
query_c_s ="""
MATCH (type:Type:Java:Experiment)
WITH type.fqn AS class
OPTIONAL MATCH (:Type:Java:Experiment{fqn:class})-[out:CALLS]->(callee:Type:Experiment)
WITH class, COUNT(DISTINCT callee.fqn) AS import
OPTIONAL MATCH (caller:Type:Experiment)-[in:CALLS]->(:Type:Java:Experiment{fqn:class})
WITH class, import, COUNT(DISTINCT caller.fqn) AS export
WHERE import + export > 0
RETURN class, import, export, SUM(import + export) AS combined
ORDER BY class
"""
df_c_s = pd.DataFrame(graph.run(query_c_s).data())

# Query coupling degrees from weighted dynamic dependency graph on class level
query_c_w ="""
MATCH (type:Type:Kieker:Experiment)
OPTIONAL MATCH (type)-[out:CALLS]->(:Type:Kieker:Experiment)
WITH type, SUM(out.weight) AS import
OPTIONAL MATCH (:Type:Kieker:Experiment)-[in:CALLS]->(type)
WITH type, import, SUM(in.weight) as export
WHERE import + export > 0
RETURN type.fqn AS class, import, export, (import+export) AS combined
ORDER BY class
"""
df_c_w = pd.DataFrame(graph.run(query_c_w).data())

# Compare coupling orders
c_s_w_i = kendalltau_distance(df_c_s, df_c_w, 'import', 'class')
c_s_w_e = kendalltau_distance(df_c_s, df_c_w, 'export', 'class')
c_s_w_c = kendalltau_distance(df_c_s, df_c_w, 'combined', 'class')

print('replication c : s <-> w: import =', format(round(c_s_w_i, 2), '.2f'), 'export =', format(round(c_s_w_e, 2), '.2f'),
      'combined =', format(round(c_s_w_c, 2), '.2f'), 'average =', format(round((c_s_w_i + c_s_w_e + c_s_w_c)/3, 2), '.2f'))

# original    (1) c : s <-> w: import = 0.36* export = 0.41* combined = 0.41* average = 0.39
# replication (1) c : s <-> w: import = 0.37 export = 0.40 combined = 0.40 average = 0.39
# original    (2) c : s <-> w: import = 0.36 export = 0.43 combined = 0.41 average = 0.40
# replication (2) c : s <-> w: import = 0.37 export = 0.42 combined = 0.40 average = 0.40
# original    (3) c : s <-> w: import = 0.42 export = 0.40 combined = 0.40 average = 0.41
# replication (3) c : s <-> w: import = 0.43 export = 0.40 combined = 0.41 average = 0.41
# original    (4) c : s <-> w: import = 0.42 export = 0.40 combined = 0.40 average = 0.41
# replication (4) c : s <-> w: import = 0.42 export = 0.40 combined = 0.41 average = 0.41

### $c : u \leftrightarrow w$

In [ ]:
# Query coupling degrees from unweighted dynamic dependency graph on class level
query_c_u ="""
MATCH (type:Type:Kieker)
OPTIONAL MATCH (type)-[out:CALLS]->(:Type:Kieker)
WITH type, COUNT(out) AS import
OPTIONAL MATCH (:Type:Kieker)-[in:CALLS]->(type)
WITH type, import, COUNT(in) as export
WHERE import + export > 0
RETURN type.fqn AS class, import, export, (import+export) AS combined
"""
df_c_u = pd.DataFrame(graph.run(query_c_u).data())

# Query coupling degrees from weighted dynamic dependency graph on class level
query_c_w ="""
MATCH (type:Type:Kieker)
OPTIONAL MATCH (type)-[out:CALLS]->(:Type:Kieker)
WITH type, SUM(out.weight) AS import
OPTIONAL MATCH (:Type:Kieker)-[in:CALLS]->(type)
WITH type, import, SUM(in.weight) as export
WHERE import + export > 0
RETURN type.fqn AS class, import, export, (import+export) AS combined
"""
df_c_w = pd.DataFrame(graph.run(query_c_w).data())

# Compare coupling orders
c_u_w_i = kendalltau_distance(df_c_u, df_c_w, 'import', 'class')
c_u_w_e = kendalltau_distance(df_c_u, df_c_w, 'export', 'class')
c_u_w_c = kendalltau_distance(df_c_u, df_c_w, 'combined', 'class')

print('replication c : u <-> w: import =', format(round(c_u_w_i, 2), '.2f'), 'export =', format(round(c_u_w_e, 2), '.2f'),
      'combined =', format(round(c_u_w_c, 2), '.2f'), 'average =', format(round((c_u_w_i + c_u_w_e + c_u_w_c)/3, 2), '.2f'))

# original    (1) c : u <-> w: import = 0.13 export = 0.24 combined = 0.29 average = 0.22
# replication (1) c : u <-> w: import = 0.12 export = 0.24 combined = 0.28 average = 0.21
# original    (2) c : u <-> w: import = 0.14 export = 0.26 combined = 0.31 average = 0.24
# replication (2) c : u <-> w: import = 0.14 export = 0.25 combined = 0.30 average = 0.23
# original    (3) c : u <-> w: import = 0.12 export = 0.22 combined = 0.28 average = 0.21
# replication (3) c : u <-> w: import = 0.11 export = 0.22 combined = 0.28 average = 0.21
# original    (4) c : u <-> w: import = 0.12 export = 0.23 combined = 0.29 average = 0.21
# replication (4) c : u <-> w: import = 0.12 export = 0.23 combined = 0.29 average = 0.21

### $p : s \leftrightarrow u$

In [ ]:
# Query coupling degrees from static dependency graph on package level
query_p_s ="""
MATCH (package:Package:Java:Experiment)
WITH package.fqn as package
OPTIONAL MATCH (:Package:Java{fqn:package})-[out:CALLS]->(callee:Package:Experiment)
WITH package, COUNT(DISTINCT callee.fqn) AS import
OPTIONAL MATCH (caller:Package:Experiment)-[in:CALLS]->(:Package:Java{fqn:package})
WITH package, import, COUNT(DISTINCT caller.fqn) AS export
WHERE import + export > 0
RETURN package, import, export, SUM(import + export) AS combined
ORDER BY package
"""
df_p_s = pd.DataFrame(graph.run(query_p_s).data())

# Query coupling degrees from unweighted dynamic dependency graph on package level
query_p_u ="""
MATCH (package:Package:Kieker:Experiment)
OPTIONAL MATCH (package)-[out:CALLS]->(:Package:Kieker:Experiment)
WITH package, COUNT(out) AS import
OPTIONAL MATCH (:Package:Kieker:Experiment)-[in:CALLS]->(package)
WITH package, import, COUNT(in) as export
WHERE import + export > 0
RETURN package.fqn AS package, import, export, (import+export) AS combined
ORDER BY package
"""
df_p_u = pd.DataFrame(graph.run(query_p_u).data())

# Compare coupling orders
p_s_u_i = kendalltau_distance(df_p_s, df_p_u, 'import', 'package')
p_s_u_e = kendalltau_distance(df_p_s, df_p_u, 'export', 'package')
p_s_u_c = kendalltau_distance(df_p_s, df_p_u, 'combined', 'package')

print('replication p : s <-> u: import =', format(round(p_s_u_i, 2), '.2f'), 'export =', format(round(p_s_u_e, 2), '.2f'),
      'combined =', format(round(p_s_u_c, 2), '.2f'), 'average =', format(round((p_s_u_i + p_s_u_e + p_s_u_c)/3, 2), '.2f'))

# original    (1) p : s <-> u: import = 0.33 export = 0.30 combined = 0.29 average = 0.31
# replication (1) p : s <-> u: import = 0.33 export = 0.30 combined = 0.29 average = 0.31
# original    (2) p : s <-> u: import = 0.31 export = 0.30 combined = 0.28 average = 0.30
# replication (2) p : s <-> u: import = 0.31 export = 0.30 combined = 0.28 average = 0.30
# original    (3) p : s <-> u: import = 0.37 export = 0.28 combined = 0.30 average = 0.32
# replication (3) p : s <-> u: import = 0.37 export = 0.28 combined = 0.30 average = 0.32
# original    (4) p : s <-> u: import = 0.36 export = 0.28 combined = 0.30 average = 0.31
# replication (4) p : s <-> u: import = 0.36 export = 0.28 combined = 0.30 average = 0.31

### $p : s \leftrightarrow w$

In [ ]:
# Query coupling degrees from static dependency graph on package level
query_p_s ="""
MATCH (package:Package:Java:Experiment)
WITH package.fqn as package
OPTIONAL MATCH (:Package:Java{fqn:package})-[out:CALLS]->(callee:Package:Experiment)
WITH package, COUNT(DISTINCT callee.fqn) AS import
OPTIONAL MATCH (caller:Package:Experiment)-[in:CALLS]->(:Package:Java{fqn:package})
WITH package, import, COUNT(DISTINCT caller.fqn) AS export
WHERE import + export > 0
RETURN package, import, export, SUM(import + export) AS combined
ORDER BY package
"""
df_p_s = pd.DataFrame(graph.run(query_p_s).data())

# Query coupling degrees from weighted dynamic dependency graph on package level
query_p_w ="""
MATCH (package:Package:Kieker:Experiment)
OPTIONAL MATCH (package)-[out:CALLS]->(:Package:Kieker:Experiment)
WITH package, SUM(out.weight) AS import
OPTIONAL MATCH (:Package:Kieker:Experiment)-[in:CALLS]->(package)
WITH package, import, SUM(in.weight) as export
WHERE import + export > 0
RETURN package.fqn AS package, import, export, (import+export) AS combined
ORDER BY package
"""
df_p_w = pd.DataFrame(graph.run(query_p_w).data())

# Compare coupling orders
p_s_w_i = kendalltau_distance(df_p_s, df_p_w, 'import', 'package')
p_s_w_e = kendalltau_distance(df_p_s, df_p_w, 'export', 'package')
p_s_w_c = kendalltau_distance(df_p_s, df_p_w, 'combined', 'package')

print('replication p : s <-> w: import =', format(round(p_s_w_i, 2), '.2f'), 'export =', format(round(p_s_w_e, 2), '.2f'),
      'combined =', format(round(p_s_w_c, 2), '.2f'), 'average =', format(round((p_s_w_i + p_s_w_e + p_s_w_c)/3, 2), '.2f'))

# original    (1) p : s <-> w: import = 0.36 export = 0.32 combined = 0.33 average = 0.33
# replication (1) p : s <-> w: import = 0.35 export = 0.31 combined = 0.32 average = 0.33
# original    (2) p : s <-> w: import = 0.35 export = 0.33 combined = 0.33 average = 0.33
# replication (2) p : s <-> w: import = 0.35 export = 0.33 combined = 0.32 average = 0.33
# original    (3) p : s <-> w: import = 0.39 export = 0.31 combined = 0.33 average = 0.35
# replication (3) p : s <-> w: import = 0.39 export = 0.31 combined = 0.33 average = 0.35
# original    (4) p : s <-> w: import = 0.39 export = 0.32 combined = 0.33 average = 0.35
# replication (4) p : s <-> w: import = 0.39 export = 0.32 combined = 0.33 average = 0.35

### $p : u \leftrightarrow w$

In [ ]:
# Query coupling degrees from unweighted dynamic dependency graph on package level
query_p_u ="""
MATCH (package:Package:Kieker)
OPTIONAL MATCH (package)-[out:CALLS]->(:Package:Kieker)
WITH package, COUNT(out) AS import
OPTIONAL MATCH (:Package:Kieker)-[in:CALLS]->(package)
WITH package, import, COUNT(in) as export
WHERE import + export > 0
RETURN package.fqn AS package, import, export, (import+export) AS combined
"""
df_p_u = pd.DataFrame(graph.run(query_p_u).data())

# Query coupling degrees from weighted dynamic dependency graph on package level
query_p_w ="""
MATCH (package:Package:Kieker)
OPTIONAL MATCH (package)-[out:CALLS]->(:Package:Kieker)
WITH package, SUM(out.weight) AS import
OPTIONAL MATCH (:Package:Kieker)-[in:CALLS]->(package)
WITH package, import, SUM(in.weight) as export
WHERE import + export > 0
RETURN package.fqn AS package, import, export, (import+export) AS combined
"""
df_p_w = pd.DataFrame(graph.run(query_p_w).data())

# Compare coupling orders
p_u_w_i = kendalltau_distance(df_p_u, df_p_w, 'import', 'package')
p_u_w_e = kendalltau_distance(df_p_u, df_p_w, 'export', 'package')
p_u_w_c = kendalltau_distance(df_p_u, df_p_w, 'combined', 'package')

print('replication p : u <-> w: import =', format(round(p_u_w_i, 2), '.2f'), 'export =', format(round(p_u_w_e, 2), '.2f'),
      'combined =', format(round(p_u_w_c, 2), '.2f'), 'average =', format(round((p_u_w_i + p_u_w_e + p_u_w_c)/3, 2), '.2f'))

# original    (1) p : u <-> w: import = 0.08 export = 0.21 combined = 0.23 average = 0.17
# replication (1) p : u <-> w: import = 0.08 export = 0.20 combined = 0.22 average = 0.17
# original    (2) p : u <-> w: import = 0.09 export = 0.22 combined = 0.23 average = 0.18
# replication (2) p : u <-> w: import = 0.08 export = 0.21 combined = 0.23 average = 0.18
# original    (3) p : u <-> w: import = 0.06 export = 0.20 combined = 0.23 average = 0.17
# replication (3) p : u <-> w: import = 0.06 export = 0.20 combined = 0.24 average = 0.17
# original    (4) p : u <-> w: import = 0.06 export = 0.20 combined = 0.24 average = 0.17
# replication (4) p : u <-> w: import = 0.06 export = 0.20 combined = 0.24 average = 0.17